# Frank Asket's Week 1 Exercise

[Frank Asket](https://github.com/frank-asket) — *Founder & CTO building Human-Centered AI infrastructure.*

To demonstrate familiarity with the OpenAI API and Ollama, this notebook is a **technical Q&A tool**: you ask a question and get an explanation (GPT with streaming, then optionally Llama). A tool you can use throughout the course.

In [2]:
# imports

import os
import sys
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

# Part 2 (website summarizer) needs scraper. Run from repo root or from week1/community-contributions/asket:
for _path in ('week1', os.path.join(os.getcwd(), '..', '..', 'week1'), os.path.join(os.getcwd(), '..', '..')):
    _p = os.path.abspath(_path)
    if _p not in sys.path and os.path.isdir(_p):
        sys.path.insert(0, _p)
        break
from scraper import fetch_website_contents

In [3]:
# constants

MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2:3b-instruct-q4_0"  # or llama3.2:1b-instruct-q4_0; run 'ollama list' to see installed models

In [4]:
# set up environment

load_dotenv(override=True)
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
if openrouter_api_key:
    openai = OpenAI(api_key=openrouter_api_key, base_url="https://openrouter.ai/api/v1")
elif openai_api_key:
    openai = OpenAI(api_key=openai_api_key)
else:
    openai = OpenAI()

In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get(\"author\") for book in books if book.get(\"author\")}
"""

In [6]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs."
user_prompt = "Please give a detailed explanation to the following question: " + question

In [7]:
# messages

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [8]:
# Get gpt-4o-mini to answer, with streaming

def strip_code_fence(text):
    """Remove only code-fence wrappers (e.g. ```markdown / ```) so prose containing 'markdown' is unchanged."""
    s = text
    if s.startswith("```markdown"):
        i = s.find("\n")
        s = s[i + 1:] if i != -1 else s[11:]
    elif s.startswith("```"):
        i = s.find("\n")
        s = s[i + 1:] if i != -1 else s[3:]
    if s.rstrip().endswith("```"):
        s = s[:s.rstrip().rfind("```")].rstrip()
    return s

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ""
    response_clean = strip_code_fence(response)
    update_display(Markdown(response_clean), display_id=display_handle.display_id)

The line of code you provided utilizes a combination of Python's generator functions and set comprehensions. Let's break it down step by step:

### Breakdown of the Code

1. **Set Comprehension**: 
   python
   {book.get("author") for book in books if book.get("author")}
   
   - This portion of the code is a set comprehension that creates a set of authors from a collection of `books`.
   - **Iteration**: It iterates over each `book` in the iterable `books`.
   - **Getting Author**: For each `book`, it attempts to retrieve the "author" using `book.get("author")`. The `get` method of a dictionary returns the value associated with the specified key, in this case, "author". If the key doesn't exist, it returns `None`.
   - **Conditional Filtering**: The expression includes a condition `if book.get("author")`, meaning that it only includes the author's name in the set if it's not `None` or an empty string (falsy values).
   - **Set Creation**: The resulting set will only contain unique authors, as sets do not allow duplicate values.

2. **Yield from**:
   python
   yield from ...
   
   - The `yield from` statement is used to yield all values from an iterable (in this case, the set created from the comprehension) one by one.
   - This means that the code will act like a generator, production one author at a time when iterated over.
   - When you call the generator function that contains this line, it will yield each unique author found in `books`.

### Purpose of the Code

- **Unique Author Extraction**: The primary purpose of this line is to extract and yield unique authors from a list (or any iterable) of books. It allows consumers of this generator to retrieve authors lazily, meaning that the authors are generated on-the-fly and you don’t need to build the entire list of authors in memory at once.
- **Efficiency**: Using `yield from` in combination with a set comprehension is efficient in terms of both time complexity (faster uniqueness management) and space complexity (not storing intermediate lists/updating for each author).

### Use Cases

- **Data Processing**: This code could be used in contexts where you want to collect authors for analysis, reports, or transformations, such as creating a summary of authors in a data processing pipeline.
- **Memory Efficiency**: It is particularly beneficial when dealing with large datasets where storing all authors’ names at once may not be feasible, but processing them one at a time is manageable.

### Example

Here’s a simple example of how this could be used:

python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C"},  # No author
    {"title": "Book D", "author": "Author 1"},  # Duplicate author
]

def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example usage
for author in unique_authors(books):
    print(author)

# Output:
# Author 1
# Author 2


In this example, the `unique_authors` generator function will produce only the unique authors of the books, omitting any duplicate entries and those books without an author.

In [16]:
# [Optional] Get Llama 3.2 to answer (requires Ollama running locally)
# If you see 'llama runner process has terminated: exit status 2' or Metal/MLX errors, upgrade Ollama from https://ollama.com/download (your 0.6.5 is old). The GPT part above is enough for the exercise.

for model_tag in ("llama3.2:1b-instruct-q4_0", "llama3.2:3b-instruct-q4_0"):
    try:
        response = ollama.chat(model=model_tag, messages=messages)
        reply = response["message"]["content"]
        display(Markdown(reply))
        break
    except Exception as e:
        if "llama3.2:3b" in model_tag:
            print("Ollama failed for both models:", e)
            print("Fix: Install the latest Ollama from https://ollama.com/download (old versions can crash on macOS). You can skip this cell; the GPT answer above is enough for the exercise.")
        continue

This code snippet is using the built-in `yield from` keyword in Python, which allows you to yield values from an iterable (such as a list or tuple) and returns a generator expression instead of creating multiple temporary variables.

Here's what the code does:

- It takes a list of books (`books`) and each book object (`book`).
- For each book in `books`, it checks if the author is present in any of the books' author lists using another method (not shown in this snippet). This is done for each book in the `books` list.
- If an author is found in a book's author list, that value is yielded from the generator expression (`yield from`). The values are directly returned by the function without creating temporary variables.

This code essentially does the following:

1. It fetches book metadata (author) for each book in the `books` list.
2. For each book, it checks if an author is present in any of its own author lists. This means it might also return values from books that don't have a specific author listed.
3. If an author is found, it yields the value directly.

The purpose of this code could be to process or filter the data in some way without storing all values in memory at once. Here are a few possible scenarios:

- Using the generated list of authors for analysis or further processing, such as filtering books based on specific authors.
- Storing only unique authors and not duplicate values from different books.

In general, using `yield from` here would allow you to process data without loading it all into memory at once. For example, you could write a function that processes each book's metadata (author, title, etc.) without having to load the entire dataset in memory.

Here is an example with some added comments for clarity:

```python
def get_book_info():
    # Fetch book metadata (author)
    books = your_list_of_books  # Replace with your actual data structure
    # Iterate over each book and check if its author list contains another book's author
    for book in books:
        author_in_other_book = False
        
        # Check if other book's author is present in current book's metadata
        for existing_book in books:
            if book.get("author") == existing_book.get("author"):
                print(f"Found a common author: {book['title']} by {existing_book['author']}")
                yield from [existing_book["author"]]  # Yield the found author
        
        # If an author is found, return it directly
        if book.get("author"):
            yield from [book.get("author")]

# Example usage:
get_book_info()
```

In this example, `get_book_info()` function fetches metadata for a list of books and uses `yield from` to iterate over each book's author list. When an author is found in another book's author list, it yields the value directly; otherwise, it returns the author.

Note: This snippet assumes you're using Python 3.7+ due to the use of `yield from` which was introduced in that version. If you're using an older version of Python or have issues with this syntax, consider upgrading your Python environment if necessary.

---
### Example: the kind of explanation this tool produces (Frank Asket)

For the question *"Explain: yield from {book.get(\"author\") for book in books if book.get(\"author\")}"*, here's a breakdown:

1. **`book.get(\"author\")`** — Retrieves the author for each book in `books` (assumed to be a list of dicts). Returns `None` if the key is missing.

2. **`{ ... for book in books if book.get(\"author\") }`** — A **set comprehension** (not a generator): builds a set of unique author names, skipping books with no author.

3. **`yield from`** — Delegates to that iterable and yields each item one by one. So the surrounding function is a **generator** that yields each unique author.

**In short:** The line is a generator that yields every unique author name from a list of book dicts, ignoring missing authors. (Note: the expression uses a set `{ }`, so it's evaluated fully before `yield from`; a memory-lighter variant would use a generator expression in parentheses.)

---
## Part 2: Website summarizer (Ollama, bilingual: English + Guéré)

Uses **Ollama only** (no API key). Fetches a URL (e.g. [Frank Asket's GitHub](https://github.com/frank-asket)) and produces a **description/summary in English** plus a version in **Guéré** (Guere), an Ivorian local language (bullet points, separated by `<hr>`). Run from repo root; ensure Ollama is installed and run `ollama serve` (and `ollama pull llama3.2` if needed).

In [ ]:
# Pull Ollama model (run once)
!ollama pull llama3.2

In [ ]:
# Check Ollama is running
try:
    requests.get("http://localhost:11434", timeout=2)
    print("Ollama is running.")
except Exception:
    print("Ollama is not running. In a terminal run: ollama serve")
    print("Then: ollama pull llama3.2")

In [ ]:
# Ollama client (OpenAI-compatible API, local)
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama_client = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

In [20]:
# Ensure scraper is importable (run from repo root or week1/community-contributions/asket)
import os, sys
for _path in ('week1', os.path.join(os.getcwd(), '..', '..', 'week1'), os.path.join(os.getcwd(), '..', '..')):
    _p = os.path.abspath(_path)
    if _p not in sys.path and os.path.isdir(_p):
        sys.path.insert(0, _p)
        break
from scraper import fetch_website_contents
from openai import OpenAI
from IPython.display import Markdown, display

# Ollama client (so this cell can run standalone)
ollama_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Prompts and summarizer (bilingual: English description + Guéré bullets, separated by <hr>)
SUMMARY_SYSTEM = """Your job is to analyze the content of a webpage and give a clear description or summary of the person or topic.
Use bullet points and keep it clear. Do not wrap the output in a code block."""
SUMMARY_USER = """Here are the contents of a webpage (e.g. a GitHub or profile page).
Extract and summarize the key details or description about the person (e.g. name, role, bio, projects). Provide an English version as a short blog-style summary with an h1 title. Then provide a second version in Guéré (Guere), an Ivorian local language from Côte d'Ivoire, with bullet points.
Separate the English and Guéré sections with an <hr> tag."""

def messages_for_site(website_text):
    return [
        {"role": "system", "content": SUMMARY_SYSTEM},
        {"role": "user", "content": SUMMARY_USER + "\n\n" + website_text}
    ]

def summarize_site(url):
    web = fetch_website_contents(url)
    # Use same tag as Part 1; run 'ollama list' to see your model name (e.g. llama3.2:3b-instruct-q4_0)
    r = ollama_client.chat.completions.create(model="llama3.2:3b-instruct-q4_0", messages=messages_for_site(web))
    return r.choices[0].message.content

def display_site_summary(url):
    display(Markdown(summarize_site(url)))

display_site_summary("https://github.com/frank-asket")

**English Summary**

Meet Franck Olivier Alex Asket, a talented developer with expertise in AI code creation. His GitHub profile showcases his contributions to various projects, including:

* AI CODE CREATION: He's proficient in tools like GitHub Copilot, GitHub Spark, and GitHub Models.
* DEVELOPER WORKFLOWS: He automates workflows using Actions and Codespaces.
* APPLICATION SECURITY: He prioritizes security with features like GitHub Advanced Security and Secret protection.

His interests lie at the intersection of software development, DevOps, and AI. With a background in healthcare, financial services, and manufacturing industries, Franck brings valuable experience to his coding endeavors.

---

**Summarize avec Guéré (English version will be provided alongside)**

Asepé frank-asket: frannk Olivier Alex Askét ( Côte d'Ivoire )

Bilangan èdey asezika:

• Ayi nanò akonnan asekan nan nana na akokon
• Akonnan à yon kounyè nan AI 
    nan akòn nou, copilot, spark an nan modeèles nan akèzè

Bilangan projeks asepé frank-asket:

* Développ' sa rèyon akonnan nan akòt nan projekts an nan
• Akonnan nan akèske nan tseksè nan koutan 
    akwa akòn nan akòn nan n'anana nan oun

Asepé nan kewòlan asepé frank-asket:

* Akonnan  akòtin nou akèmnon nan koutou nan akòn
* Nou akòn nan akèske nan ansa nan sekanan 
   nan oublò akwa nan akòn nan nan nana

**hr>**

---

English summary remains the same